## filtered csv common_titles

it creates 2 dictionaries:
* normalized titles and the number of appearences
* normalized titles and its 1000 most common tracks

In [1]:
"""
    usage:

        python 2_common_titles_and_tracks_maker.py path-to-mpd-data/
"""



import pandas as pd
import numpy as np
import sys
import json
import re
from collections import Counter, OrderedDict
import os## filtered csv common_titles
import datetime
import csv
from gensim.models import Word2Vec
from tqdm import tqdm_notebook
from whoosh.analysis import CharsetFilter, StemmingAnalyzer
from whoosh import fields
from whoosh.support.charset import accent_map
import json
import re
import pandas as pd
import numpy as np
from whoosh.analysis import CharsetFilter, StemmingAnalyzer
from whoosh import fields
from whoosh.support.charset import accent_map
from nltk.stem import PorterStemmer
from string import ascii_letters

total_playlists = 0
title_histogram = Counter()
dict_sorted_titles = dict()

quick = False
max_files_for_quick_processing = 1


def normalize_name(name):
    stem = True
  
    letters = list(name)
    
    # if format w o r k o u t / w.o.r.k.o.u.t/ w*o*r*k*o*u*t join togother
    if len(letters)>4:
        if len(set([letters[i] for i in range(0,len(letters),2)]))==1:
            name = "".join([letters[i] for i in range(1,len(letters),2)])
        elif len(set([letters[i] for i in range(1,len(letters),2)]))==1:
            name = "".join([letters[i] for i in range(0,len(letters),2)])
             
    # if there is and & not surrounded by spaces, leave alone (example 'r&b)
    if "&" in letters:
        position = letters.index("&")
        if position>0 and position<len(letters)-1:
            if letters[position-1]!=' ' and letters[position+1]!=' ':
                stem  = False
    
      
    # if there is a k surrounded by numbers turn to 0
    if "k" in letters and '2' in letters:
        positions = [x for x in range(len(letters)) if letters[x]=='k']
        for pos in positions:
             if pos>0 and pos<len(letters)-1:
                if letters[pos-1]=='2':
                    letters[pos]='0'
                    name = "".join(letters)
           
    # proceed to stem   
    if stem: 
        my_analyzer = StemmingAnalyzer() | CharsetFilter(accent_map)
        tokens = my_analyzer(name)
        words = [token.text for token in tokens]
        
        # if the reuslt is empyt, leave alone, if not, return as a list
        if len(words)!=0:
            result=""
            for el in words:
                result +=el+" "
            letters = list(result)[:-1]
    # softer stem
    else:
        name = name.lower()
        name = re.sub(r"[.,'\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
        name = re.sub(r'\s+', ' ', name).strip()
        letters = list(name)
        
            
            
    # if last n characters are equal leave only 1 
    last = letters[-1]
    if last in ascii_letters and len(letters)>1:
        while(letters[-2]==last):
            letters.pop(-2)
            if len(letters)==1: break
    
    
    return ''.join(letters)



def process_mpd(path):
    print("making dictionary of titles...")
    create_common_titles(path)
    reorder()
    print("filling dictionary of titles...")
    fill_common_titles(path)
    print("cuting to best 1000 tracks...")
    cut_1000()   
    np.save('dictionaries/ntitles_1000tracks_final', dict_sorted_titles) 
    print("dictionaries successfully saved")
    show_summary()

def create_common_titles(path):
    global total_playlists
    count = 0
    filenames = os.listdir(path)
    for filename in tqdm_notebook(sorted(filenames)):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                process_playlist(playlist)
                total_playlists+=1
            count += 1

            if quick and count > max_files_for_quick_processing:
                break
                
def reorder():
    global dict_sorted_titles
    if not os.path.exists("dictionaries"):
        os.makedirs("dictionaries")
    dict_sorted_titles = OrderedDict(title_histogram.most_common())
    np.save('dictionaries/dict_ntitles_final', dict_sorted_titles)
    for word in dict_sorted_titles:
        dict_sorted_titles[word]=Counter()
    
        
        
def fill_common_titles(path):
    global dict_sorted_titles
    count = 0
    filenames = os.listdir(path)
    for filename in tqdm_notebook(sorted(filenames)):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                title = normalize_name(playlist["name"])
                if title in dict_sorted_titles.keys():
                    for track in playlist["tracks"]:
                        dict_sorted_titles[title][track["track_uri"]] += 1    
                else: print(title)
                             
            count += 1

            if quick and count > max_files_for_quick_processing:
                break
                
def cut_1000():
    
     # cut to 1000 best tracks 
    for title in dict_sorted_titles:
        if type(dict_sorted_titles[title])!=Counter:
            print(title,type(dict_sorted_titles[title]))
        else:
            dict_sorted_titles[title] = dict_sorted_titles[title].most_common(1000)
            #genre_tracks[ti] = [l[0] for l in genre_tracks[ti]]
            #if len(genre_tracks[ti])<600:
            #    not_enough+=1 
    

def show_summary():
    print ("number of playlists", total_playlists)
    print ("number of unique normalized titles", len(dict_sorted_titles))
        


def process_playlist(playlist):
    nname = normalize_name(playlist['name'])
    title_histogram[nname] += 1


if __name__ == '__main__':
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == '--quick':
        quick =  False
    process_mpd('..')
    

making dictionary of titles...


FileNotFoundError: [Errno 2] No such file or directory: '-f'